In [1]:
#при использовании GPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [2]:
#зависимости для использования моделей с HF
!pip install transformers
from transformers import pipeline, AutoTokenizer, EncoderDecoderModel

!pip install sentencepiece
import sentencepiece

In [3]:
#перевод ru-eng (HF)
tokenizer_tr = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
tr_pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-ru-en", tokenizer = tokenizer_tr, device = device)

source.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [4]:
#модель суммаризации (HF)
pipe = pipeline("summarization", model="facebook/bart-large-cnn", device = device)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
#перевод eng-ru (HF)
pipe_en_ru = pipeline("translation", model="Helsinki-NLP/opus-mt-en-ru",  device = device)

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

In [6]:
#подключение гугл-диска
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#зависимости
import pandas as pd
import numpy as np

from tqdm import tqdm

import re

#исправление орфографических ошибок
!pip install pyaspeller
from pyaspeller import YandexSpeller

#убираем стоп-слова
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#лемматизация
!pip install pymorphy2
import pymorphy2

#для автоматической загрузки файлов
from google.colab import files

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 46.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=9df26896b9a01d9833210813b96c4be47ac856fdef9ad1b3a5cdbf9b995c1deb
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [8]:
#загрузка данных (посты и комментарии)
data = pd.read_json('/content/drive/MyDrive/BrandAnalitics/dataset.jsonl', lines=True)
data

,text,url,id,hash,date,root_id,parent_id
0,Наша многодетная семья переехала из Калифорнии...,http://www.youtube.com/watch?v=_UYt9yJFck0,_UYt9yJFck0,008c23470d1454e362218325921370f1,2023-11-05 00:59:59,NaN,NaN
1,Прохождение DEAD SPACE на русском языке.,http://www.youtube.com/watch?v=ZZDu5IRiI9g,ZZDu5IRiI9g,0051fdc65fa052f78f1f616ea29952b3,2023-11-04 09:00:23,NaN,NaN
2,Рацион для похудения на 2500рублей в неделю,http://www.youtube.com/watch?v=VJ0dk0dxRRk,VJ0dk0dxRRk,008d75c75c26965a7a16515905a87084,2023-11-04 20:46:27,NaN,NaN
3,Всем доброго времени суток <br />\nВот такая л...,http://www.youtube.com/watch?v=Irzr95K6T9A,Irzr95K6T9A,00121ecaceac7de50318003e5962357c,2023-11-04 12:51:19,NaN,NaN
4,Как вам такое видео)💕 @,http://www.youtube.com/watch?v=9-GWe3L8efY,9-GWe3L8efY,007310caf9f4e257dafc0981bbc6cc70,2023-11-05 08:30:34,NaN,NaN
...,...,...,...,...,...,...,...
443893,"Ирина Антошина, да, для разнообразия )",https://dzen.ru/a/ZWx-YFcgKRpYNNZL#comment_195...,-9050536880625669250_1950779633,84aa7d0e06a175f983a44899cb436fcb,2023-12-04 08:37:34,-9050536880625669250,-9050536880625669250_1950501953
443894,"Ольга Jhkjdf, ну вот, а мне без кота приходить...",https://dzen.ru/a/ZWx-YFcgKRpYNNZL#comment_195...,-9050536880625669250_1950698203,b27b4c84d166cf1fb1b831143b6340b9,2023-12-04 07:34:55,-9050536880625669250,-9050536880625669250_1950525243
443895,"Nice&Easy, 😘😘😘",https://dzen.ru/a/ZWx-YFcgKRpYNNZL#comment_195...,-9050536880625669250_1950716233,47014ab29b611fa67aaf7bc758e6935e,2023-12-04 07:48:27,-9050536880625669250,-9050536880625669250_1950537643
443896,"Светлана Олейник, Светуль, какие прелестные де...",https://dzen.ru/a/ZWx-YFcgKRpYNNZL#comment_195...,-9050536880625669250_1950782183,a889444669c5fe5d6ffdd221caf26661,2023-12-04 08:39:24,-9050536880625669250,-9050536880625669250_1950535563


##Примеры постов и комментов

In [ ]:
data['text'][0].replace('<br />\n', ' ')

'Наша многодетная семья переехала из Калифорнии в Северную Каролину. Здесь мы делимся нашими буднями, нашими покупками, прогулками, путешествиями, а также вкусными рецептами.  Подписывайтесь на наш канал, чтобы не пропустить новые видео. Мы очень рады каждому из вас!  Наша Жизнь в Штатах - https://www.youtube.com/channel/UCp1j... Yandex Zen - https://zen.yandex.ru/id/622acfe69c6b...  Instagram -  https://www.instagram.com/valeriya_america/  Нажимайте на колокольчик, чтобы получать уведомления о новых видео.  КАНАЛ МИИ — https://www.youtube.com/@miaslifeadventures4106    Канал с Музыкой из Моих Видео  -  https://www.youtube.com/@tokyomusicwalker4038   Ссылка на оригинальный рецепт - https://youtu.be/ag55w2vd9mM?si=pIFRK0nZHZ7MgDIV  ПИРОГ  С ГРУШАМИ 400 г - груши 50 г - сахар 20 г - сливочное масло 1 ч л - корица (по желанию)  Тесто 200 г - мука 12 г - разрыхлитель 3 - яйца 100 г - сахар 1/4 ч л - соль 1 ч л - ванильеый экстракт 100 мл - молоко 100 мл - растительное масло  * Диаметр форм

In [ ]:
data['text'][150].replace('<br />\n', ' ')

'Очередная передача цикла посвящена ответам на письма молодых мам, врач-педиатр, профессор Николай Тюрин объясняет, как ухаживать за заболевшим ребенком до прихода врача, педиатры Мария Цветкова, Татьяна Щенкова, Зинаида Егорова рассказывают о методиках закаливания грудных детей. В выпуске есть фрагменты без звука<br />\r\n<br />\r\nГлавная редакция научно-популярных и образовательных программ 1978<br />\r\nАвтор сценария - Николай Тюрин<br />\r\nРежиссер - Марина Хрусталева<br />\r\nОператор - Марина Голдовская<br />\r\nОператор - Тариэл Автандилян<br />\r\nВедущий - Николай Тюрин<br />\r\n<br />\r\n#МаминаШкола #вопросы #медицина<br />\r\n<br />\r\nМы в соцсетях:<br />\r\nЯндекс. Дзен - https://zen.yandex.ru/fond<br />\r\nTelegram - https://t.me/gosteleradiofond<br />\r\nVK - https://vk.com/teleradiofond<br />\r\nOK - https://ok.ru/gosteleradiofond<br />\r\n<br />\r\n▶Подписаться на канал "Советское телевидение": https://www.youtube.com/@gtrftv?sub_confirmation=1'

In [ ]:
data['text'][443897].replace('<br />\n', ' ')

'Nice&Easy, Понимаю😺мне приятно🙏Внучка маленькая ,я одна была с ней.Только вечером на Невской уехали.Были в театре на Пеппи длинный чулок👍Очень понравилось,устала конечно.'

In [ ]:
data['text'][443896].replace('<br />\n', ' ')

'Светлана Олейник, Светуль, какие прелестные девчушки! Ты счастливая бабушка!😍😘💖'

In [ ]:
data['text'][421894].replace('<br />\n', ' ')

'1'

In [ ]:
data['text'][100].replace('<br />\n', ' ')

'#никах #любовь #альхамдулиллах #сунна #семья'

In [ ]:
data['text'][1000].replace('<br />\n', ' ')

'Я скаяал скинс фарм и выполнил задания написано в теченит 24ч будет награда за выполненое задание жду 3 день'

In [ ]:
data['text'][10].replace('<br />\n', ' ')

'Концовка Call of Duty: Modern Warfare 3 вместе с секретной концовкой. Прохождение Call of Duty: Modern Warfare 3 на русском: https://www.youtube.com/playlist?list=PLkJoTOgcypA_kMS4EX3y4WfpUapR8wFq- Подписка на канал: http://bit.ly/2DB6Fhm =) Мой Telegram: https://t.me/rusgametactics VK: https://vk.com/alexander.plav Группа RusGameTactics: https://vk.com/rusgametactics Call of Duty: Modern Warfare III 2023: https://www.callofduty.com/ru/modernwarfare3  Об игре Call of Duty: Modern Warfare III (2023): Капитан Прайс и ОТГ-141 противостоят самой страшной угрозе. Ультранационалистический военный преступник Владимир Макаров распространяет свою власть по всему миру, заставляя ОТГ-141 сражаться как никогда раньше.  Прохождение Call of Duty Vanguard: https://youtu.be/rKq9I8GdDvA Прохождение Call of Duty: WW2 (World War 2): https://youtu.be/gyangAld6I0 Прохождение Watch Dogs Legion: https://youtu.be/MQS_dc8XgKA Assassin’s Creed 1: https://youtu.be/3WBCoU7aAGU  Прохождение Wolfenstein 2: The New

## Очистка данных

Убираем из текстов: <br />
1) спецсимволы пробела, переноса строки, картинки и т.п. <br />
2) ссылки на сайты, id тегнутых пользователей <br />
3) смайлы <br />

In [9]:
def clean_up (text):
  #Функция удаляет из текста спецсимвол пробела и ссылки
  text = text.replace('<br />\n', ' ').replace('\n', ' '). replace('<b>', ' ').replace('<br>', ' ').replace('</b>', ' ').replace('</i>', ' ')
  text = text.replace('<a href="', ' ').replace('</a>', ' ').replace('<i>', ' ').replace('<hr/>', ' ').replace('<br/>', ' ').replace('\r', ' ')
  text = text.replace('src="', ' ').replace('alt="', ' ').replace('img', ' ')
  text = re.sub(r"https?://[^,\s]+,?", ' ', text)
  text = re.sub("\[id\S+", ' ', text)
  #удаление смайлов
  text = re.sub('[^\x00-\x7Fа-яА-Я]', '', text)

  return text


In [10]:
data['text'] = data['text'].apply(lambda x: clean_up(x))

##Мерджим посты с комментами

In [11]:
#посты - это то, у чего пустой столбец 'root_id', комментарии - всё остальное
posts = data[data['root_id'].isna()]
comments = data[~data['root_id'].isna()]

In [12]:
#удаляем комменты, короче 10 символов, как малоинформативные

short_comments_indx = []

for ind in tqdm(comments.index):
  if len(comments.text[ind]) < 10:
    short_comments_indx.append(ind)

comments.drop(short_comments_indx, inplace = True)

100%|██████████| 433890/433890 [00:12<00:00, 35684.76it/s]
<ipython-input-12-65cd997ffeaf>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments.drop(short_comments_indx, inplace = True)


In [32]:
#исправление орфографических ошибок в постах

speller = YandexSpeller()
clean_posts=[]
for i in tqdm(range(len(posts['text']))):
  clean_posts.append(speller.spelled(posts['text'][i][:1000]))
posts['text'] = clean_posts

  5%|▍         | 500/10008 [10:55<3:27:42,  1.31s/it]


KeyError: ignored

In [ ]:
clean_comments = []
for i in tqdm(range(len(comments['text']))):
  clean_comments.append(speller.spelled(comments['text'][i][:1000]))
comments['text'] = clean_comments

100%|██████████| 6687/6687 [1:25:53<00:00,  1.30it/s]


In [ ]:
#сохраняем очищенные посты и комменты
posts.to_csv('clean_posts.csv')
files.download('/content/clean_posts.csv')
comments.to_csv('clean_comments.csv')
files.download('/content/clean_comments.csv')

In [13]:
#соотносим комментарии с постами
posts = posts.drop(columns = ['root_id']).rename(columns = {'id' : 'root_id'})
df = posts.merge(comments, on = 'root_id')
df

,text_x,url_x,root_id,hash_x,date_x,parent_id_x,text_y,url_y,id,hash_y,date_y,parent_id_y
0,Наша многодетная семья переехала из Калифорнии...,http://www.youtube.com/watch?v=_UYt9yJFck0,_UYt9yJFck0,008c23470d1454e362218325921370f1,2023-11-05 00:59:59,NaN,"Валерея доброго времени суток! Очень сожалею, ...",http://www.youtube.com/watch?v=_UYt9yJFck0&lc=...,UgxbGNB3BzRBzp51Pft4AaABAg,3c912f43f71448135b91b7a469abb409,2023-11-05 05:00:00,_UYt9yJFck0
1,Наша многодетная семья переехала из Калифорнии...,http://www.youtube.com/watch?v=_UYt9yJFck0,_UYt9yJFck0,008c23470d1454e362218325921370f1,2023-11-05 00:59:59,NaN,"Почитайте историю Хэловина, и не пишите, если ...",http://www.youtube.com/watch?v=_UYt9yJFck0&lc=...,Ugy8QWttGacfEt3X0nl4AaABAg.9wiGws53R4J9wiUgO58IW0,cad2857f538f76ca928514c084064dc9,2023-11-05 03:00:00,Ugy8QWttGacfEt3X0nl4AaABAg
2,Наша многодетная семья переехала из Калифорнии...,http://www.youtube.com/watch?v=_UYt9yJFck0,_UYt9yJFck0,008c23470d1454e362218325921370f1,2023-11-05 00:59:59,NaN,"Лера,а конфеты уже внутри?",http://www.youtube.com/watch?v=_UYt9yJFck0&lc=...,Ugzg6ptyvRMkS1QWaVF4AaABAg,7bf172306ca80672397ff474b72fce25,2023-11-05 05:00:00,_UYt9yJFck0
3,Наша многодетная семья переехала из Калифорнии...,http://www.youtube.com/watch?v=_UYt9yJFck0,_UYt9yJFck0,008c23470d1454e362218325921370f1,2023-11-05 00:59:59,NaN,"@gatta28учитесь общаться с людьми, не тыкайте....",http://www.youtube.com/watch?v=_UYt9yJFck0&lc=...,Ugy8QWttGacfEt3X0nl4AaABAg.9wiGws53R4J9wiYVObMGfF,4c772643e3ebbd01f0cf01cccdb402ba,2023-11-05 04:00:00,Ugy8QWttGacfEt3X0nl4AaABAg
4,Наша многодетная семья переехала из Калифорнии...,http://www.youtube.com/watch?v=_UYt9yJFck0,_UYt9yJFck0,008c23470d1454e362218325921370f1,2023-11-05 00:59:59,NaN,Сама хожу с переломом мизинца. Это уже не перв...,http://www.youtube.com/watch?v=_UYt9yJFck0&lc=...,Ugy2NwZNhhWamuy3zMt4AaABAg,ae1d9b9b877b2c51787184d86e8ec843,2023-11-05 02:00:00,_UYt9yJFck0
...,...,...,...,...,...,...,...,...,...,...,...,...
352381,Вот смотрели зрители в ледовых шоу как Алекс...,https://dzen.ru/a/ZWxxfnv69RavXRXH,756471475437301336,04f5dd38e29f746fd188f640bf26fd80,2023-12-03 13:06:33,NaN,"Эх, Алнка))) родись она на год и 2 месяца пора...",https://dzen.ru/a/ZWxxfnv69RavXRXH#comment_193...,756471475437301336_1939824451,1e05fcc9c504173d30873db0baf239bd,2023-12-03 14:20:21,756471475437301336
352382,Вот смотрели зрители в ледовых шоу как Алекс...,https://dzen.ru/a/ZWxxfnv69RavXRXH,756471475437301336,04f5dd38e29f746fd188f640bf26fd80,2023-12-03 13:06:33,NaN,"Василий Семакин, для открытия катка сойдет.",https://dzen.ru/a/ZWxxfnv69RavXRXH#comment_193...,756471475437301336_1939895631,998373292a6e530cda443086413f1e55,2023-12-03 15:14:25,756471475437301336_1939882131
352383,Вот смотрели зрители в ледовых шоу как Алекс...,https://dzen.ru/a/ZWxxfnv69RavXRXH,756471475437301336,04f5dd38e29f746fd188f640bf26fd80,2023-12-03 13:06:33,NaN,"Василий Семакин, а как насчт Диши? Или это дру...",https://dzen.ru/a/ZWxxfnv69RavXRXH#comment_193...,756471475437301336_1939881791,49bead7f821a1b3f6a203ae5309dfa24,2023-12-03 15:03:47,756471475437301336_1939839841
352384,Вот смотрели зрители в ледовых шоу как Алекс...,https://dzen.ru/a/ZWxxfnv69RavXRXH,756471475437301336,04f5dd38e29f746fd188f640bf26fd80,2023-12-03 13:06:33,NaN,"Нина Б,только ни кто ее уже не помнит,однодневка",https://dzen.ru/a/ZWxxfnv69RavXRXH#comment_193...,756471475437301336_1939861181,9fda18f94ca778a8a6b6609f4a0110a5,2023-12-03 14:47:46,756471475437301336_1939762271


In [14]:
#группируем все комментарии по одному посту
df['text_y'] = df['text_y']+'. '
all_comments = df.groupby('text_x').agg({'text_y' : 'sum'})

In [ ]:
all_comments

,text_y
text_x,
"1 0 - 1 1 - 2 3 Могут возникать разногласия с людьми, с которыми вы обычно отлично ладите: друзьями, коллегами, родственниками. Серьезных конфликтов не будет, вы быстро найдете решения, с которыми согласятся все. Многим Тельцам пригодится опыт, полученный раньше. Зная о нем, окружающие чаще, чем обычно, будут обращаться к представителям знака за советами или помощью. Вы постараетесь никому не отказывать, но будете помнить и о собственных интересах. Вторая половина дня будет особенно благоприятной для общения. Вы легко поладите со старыми и новыми знакомыми. Не исключена встреча с человеком, который когда-то был вам очень дорог. Восстановить отношения будет нетрудно, если вы этого захотите. Позже у вас появится возможность спокойно обдумать какие-то важные вещи, обсудить планы с теми, чьи советы уже не раз помогали. Не исключено, что появится шанс научиться чему-то новому. Вы быстро запомните все, что нужно; полученная информация вскоре пригодится. Дел будет больше, чем обычно, но вас это не напугает. Не исключено, что нужно будет решать несколько вопросов сразу, думать о многих вещах одновременно. К вечеру вы почувствуете, что устали. Но это не помешает отлично провести время с друзьями. Порадует общение с самыми близкими, а вот на многолюдных мероприятиях вы можете чувствовать дискомфорт. Вечером многим Девам захочется побыть в тишине, чтобы восстановить силы и душевное равновесие. Хорошо покупать новую мебель или бытовую технику, а также другие вещи, которые будут радовать вас и ваших близких. Вечером важно избегать перегрузок и стрессовых ситуаций. Вероятны мелкие домашние недоразумения. Бытовая техника может ломаться. Но все проблемы такого рода вы решите быстро; для этого могут потребоваться небольшие расходы. Окружающие будут вмешиваться в ваши дела чаще, чем обычно. Поначалу это может раздражать, но чуть позже вы поймете, как извлечь пользу из происходящего. Вторая половина дня порадует хорошими новостями издалека. Вторая половина дня пройдет проще и приятнее. Могут появиться новые дела и задачи, но вы быстро с ними справитесь. Вечер стоит провести с близкими. Будет приятно и полезно обсудить события последних дней, поделиться планами на будущее. Вероятны необычные встречи. Не исключено, например, что вы познакомитесь с человеком, о котором слышали много хорошего, или столкнетесь с какой-то известной личностью. Вряд ли немедленно завяжется дружба, но пообщаться будет интересно. День подойдет для общения с давними знакомыми. Наверняка найдется, о чем поговорить с ними, да и общие планы могут появиться. Вероятны денежные поступления, в том числе из неожиданных источников.",как обычно мне говно попалось. Я разбогатею ур...
"Я готова сегодня зарядится позитивной энергией. Я - самое дорогое, что есть у меня, и я ценю себя. Я готова к любым трансформациям. ДЛЯ АКТИВАЦИИ ПИШИ ТАК И ЕСТЬ",Так и есть!. Я Натали готова сегодня и всегда ...
номер в комментарии + причина ( по желанию ),1. ава лучшая..
"Я долго думал, и пришл к такому выводу. Я всех вас люблю. Вы лучшие союзники, именно те, кто остался с нами. Благодарен нашим братьям, жнам, что уже год были с нами. Нашим котикам, хорбкам, хомячкам. Дети - мы вас тоже очень любим. Хочу сказать спасибо всем союзникам, что были. Выделю пару человек, да, без этого никак. Сэм, Одри, Миран, Рин, Сок, Кими, и многие другие, вы и сами знаете. Вирт уже стал не тот, или же я ""вырос"" и мне стало не интересно. Выполняя свою работу тут, не потому что я хочу, а потому что нужно. Я просто выгорел, и мне было тяжело это признавать, но все же, это конец. Конец Ватикану. Все истории заканчиваются. И наша подошла к концу. Возможно когда нибудь мы и вернмся, но это не точно. 23.07.2022 - 13.11.2023 Нормальная такая дата, как по мне. Все союзы РАЗРЫВАЮТСЯ, рад был сотрудничать. Название за нами, если узнаю, что взяли, прибью. Шуга Фенриц #_Vs {Важно}",не могу поверить. уже такими родными стали. бе...
"Автор проделал большую работу, чтобы собрать все материалы. Ваши соображения? Причи

##Суммаризация всех комментов

In [8]:
#загрузка предобработанных данных
clean_data = pd.read_csv('/content/drive/MyDrive/BrandAnalitics/all_comments_clean.csv')
clean_data

,text_x,text_y
0,...,как обычно мне говно попалось. Я разбогатею ур...
1,Я готова сегодня заряди...,Так и есть!. Я Натали готова сегодня и всегда ...
2,номер в комментарии + причи...,1. ава лучшая..
3,"Я долго думал, и пришл к такому...",не могу поверить. уже такими родными стали. бе...
4,Автор проделал большую раб...,Это было понятно еще самого начала. Ну вот и о...
...,...,...
9928,"я сегодня словила магию чисел: 11 ноября, 11 ...","привет! пока еще нет, но походу скоро будет.. ..."
9929,"я хочу орфея, поля, я ипалил катку за орфея и ...",ОООО ЕБАТЬ. я не умею против него играть. А КА...
9930,я хочу ходить в спортзал я хочу собственными р...,я смогу выплатить все свои кредиты и быть може...
9931,"я честно говоря в шоке, ЗАВТРА камбэк..... в...","проверьте у меня пульс. ""> . казалось..."


In [ ]:
#перевод ru-eng
tr_comments = []
for comment in tqdm(clean_data['text_y']):
  if len(comment) < 512:
    tr_comments.append(str(tr_pipe(comment)[0]['translation_text']))
  else:
    part = 0
    tr = ''
    while part <= int(len(comment)/512):
      tr+=str(tr_pipe(comment[part*512 : (part+1)*512])[0]['translation_text'])
      part +=1
    tr_comments.append(tr)

  0%|          | 6/9933 [00:31<17:49:22,  6.46s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  5%|▌         | 514/9933 [18:08<4:29:13,  1.71s/it]Your input_length: 501 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 503 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 502 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
 40%|████      | 3983/9933 [2:11:00<2:37:58,  1.59s/it]

In [1]:
tr_comments.to_csv('tr_comments.csv')
files.download('tr_comments.csv')

NameError: ignored

In [18]:
#суммаризация
summary = []
for tr_comment in tqdm(tr_comments):
  summary.append(pipe(tr_comment, max_length=300, min_length=10, do_sample=False)[0]['summary_text'])

100%|██████████| 10/10 [01:54<00:00, 11.47s/it]


In [24]:
#перевод eng - ru
rus_summary = []
for text in tqdm(summary):
  rus_summary.append(pipe_en_ru(text)[0]['translation_text'])

100%|██████████| 10/10 [00:51<00:00,  5.18s/it]


In [25]:
rus_summary

['"Я стану отцом, разбогатею, и деньги упадут с неба", - говорит он. - Спасибо.',
 'Это последний день первой недели второго месяца года. Пора попрощаться с первым месяцем 2013 года.',
 'Эйва - лучшая.',
 'Не могу поверить, я тоже, дорогая, обнимаю и Ватиканские рекорды ..мои хорошие, мои любимые... .. удачи вам, ребята.',
 'Ливан на пути, в территориальных водах тоже что-то есть, Сирия, Кипр или все это не считается... 600 миллиардов долларов под ногами, и инфраструктура поднимается, получает, продает. Ливан и Турция каким-то образом вовлечены в развитие Левиафана.',
 'На e36 это было вроде как "/ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  . . . . . . . . .    . . . . . . . . . . . . . . . . .',
 '[Did693121918 minaku], спасён. [Did439487454xayaka], жду! [Did67

In [36]:
clean_data['text_y'][0]

'как обычно мне говно попалось. Я разбогатею ура. Деньги с неба упадут . Спасибо стеша. окей спасибо стеш. спасибо, проведу вечер в эдм. Спасибо стеш,я стану папой. '

##Комменты к посту - комменты о компании не к посту - левые

In [14]:
#приводим тексты к нижнему регистру, делим на слова
df['vec_text_x'] = df['text_x'].apply(lambda x: x.lower().split())
df['vec_text_y'] = df['text_y'].apply(lambda y: y.lower().split())

In [15]:
#убираем стоп-слова

stop = stopwords.words('russian')
df['vec_text_x'] = df['vec_text_x'].apply(lambda x: [word for word in x if word not in stop])
df['vec_text_y'] = df['vec_text_y'].apply(lambda y: [word for word in y if word not in stop])

In [16]:
#убираем лишние символы
for text in tqdm(df['vec_text_x']):
  for i in range(len(text)):
    text[i] = re.compile('[^а-яА-Яa-zA-Z0-9 ]').sub('', text[i])

for text in tqdm(df['vec_text_y']):
  for i in range(len(text)):
    text[i] = re.compile('[^а-яА-Яa-zA-Z0-9 ]').sub('', text[i])

100%|██████████| 352386/352386 [00:08<00:00, 42169.29it/s]


In [17]:
#лемматизация

text_x_norm =[]
text_y_norm = []
analyzer=pymorphy2.MorphAnalyzer()
for i in tqdm(range(len(df['text_x']))):
  text_x_norm.append([analyzer.parse(word)[0].normal_form for word in df['vec_text_x'][i]])
  text_y_norm.append([analyzer.parse(word)[0].normal_form for word in df['vec_text_y'][i]])
df['text_x_norm'] = text_x_norm
df['text_y_norm'] = text_y_norm

100%|██████████| 352386/352386 [1:12:02<00:00, 81.52it/s]


In [18]:
#TF & TOP50 word (posts)
top50_text_x =[]
for text in tqdm(df['text_x_norm']):
  freq_dict = {}

  for i in range(len(text)):
    freq = 1
    if text[i] not in freq_dict.keys():
      freq_dict[text[i]] = freq
      freq +=1
    else:
      freq +=1
      freq_dict[text[i]] = freq

  top50_text_x.append(sorted(freq_dict.items(), key=lambda x:x[1])[-50:])


100%|██████████| 352386/352386 [00:21<00:00, 16706.66it/s]


In [19]:
#TF & TOP50 word (comments)
top50_text_y =[]
for text in tqdm(df['text_y_norm']):
  freq_dict = {}

  for i in range(len(text)):
    freq = 1
    if text[i] not in freq_dict.keys():
      freq_dict[text[i]] = freq
      freq +=1
    else:
      freq +=1
      freq_dict[text[i]] = freq
  sorted(freq_dict.items(), key=lambda x:x[1])
  top50_text_y.append(sorted(freq_dict.items(), key=lambda x:x[1])[-50:])

100%|██████████| 352386/352386 [00:04<00:00, 73885.07it/s]


In [20]:
df['top50_text_x'] = top50_text_x
df['top50_text_y'] = top50_text_y

In [21]:
#оставляем только слова (без частоты использования в тексте)
words_x = []

for i in tqdm(range(len(df['top50_text_x']))):
  lst = []
  for j in range(len(df['top50_text_x'][i])):
    lst.append(df['top50_text_x'][i][j][0])
  words_x.append(lst)

100%|██████████| 352386/352386 [01:13<00:00, 4810.69it/s] 


In [22]:
#оставляем только слова (без частоты использования в тексте)
words_y = []

for i in tqdm(range(len(df['top50_text_y']))):
  lst = []
  for j in range(len(df['top50_text_y'][i])):
    lst.append(df['top50_text_y'][i][j][0])
  words_y.append(lst)

100%|██████████| 352386/352386 [00:21<00:00, 16335.15it/s]


In [23]:
df['top50_text_x'] = words_x
df['top50_text_y'] = words_y

In [24]:
#save as CSV file and download TOP50 tokens
df.to_csv('top50_tf.csv')
files.download('/content/top50_tf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
df

,text_x,url_x,root_id,hash_x,date_x,parent_id_x,text_y,url_y,id,hash_y,date_y,parent_id_y,vec_text_x,vec_text_y,text_x_norm,text_y_norm,top50_text_x,top50_text_y
0,Наша многодетная семья переехала из Калифорнии...,http://www.youtube.com/watch?v=_UYt9yJFck0,_UYt9yJFck0,008c23470d1454e362218325921370f1,2023-11-05 00:59:59,NaN,"Валерея доброго времени суток! Очень сожалею, ...",http://www.youtube.com/watch?v=_UYt9yJFck0&lc=...,UgxbGNB3BzRBzp51Pft4AaABAg,3c912f43f71448135b91b7a469abb409,2023-11-05 05:00:00,_UYt9yJFck0,"[наша, многодетная, семья, переехала, калифорн...","[валерея, доброго, времени, суток, очень, сожа...","[наш, многодетный, семья, переехать, калифорни...","[валерей, добрый, время, сутки, очень, сожалет...","[америкацен, америка, ценывсша, ценывамерика, ...","[валерей, добрый, время, сутки, очень, сожалет..."
1,Наша многодетная семья переехала из Калифорнии...,http://www.youtube.com/watch?v=_UYt9yJFck0,_UYt9yJFck0,008c23470d1454e362218325921370f1,2023-11-05 00:59:59,NaN,"Почитайте историю Хэловина, и не пишите, если ...",http://www.youtube.com/watch?v=_UYt9yJFck0&lc=...,Ugy8QWttGacfEt3X0nl4AaABAg.9wiGws53R4J9wiUgO58IW0,cad2857f538f76ca928514c084064dc9,2023-11-05 03:00:00,Ugy8QWttGacfEt3X0nl4AaABAg,"[наша, многодетная, семья, переехала, калифорн...","[почитайте, историю, хэловина, пишите, курсе, ...","[наш, многодетный, семья, переехать, калифорни...","[почитать, история, хэловин, писать, курс, дем...","[америкацен, америка, ценывсша, ценывамерика, ...","[почитать, история, хэловин, писать, курс, дем..."
2,Наша многодетная семья переехала из Калифорнии...,http://www.youtube.com/watch?v=_UYt9yJFck0,_UYt9yJFck0,008c23470d1454e362218325921370f1,2023-11-05 00:59:59,NaN,"Лера,а конфеты уже внутри?",http://www.youtube.com/watch?v=_UYt9yJFck0&lc=...,Ugzg6ptyvRMkS1QWaVF4AaABAg,7bf172306ca80672397ff474b72fce25,2023-11-05 05:00:00,_UYt9yJFck0,"[наша, многодетная, семья, переехала, калифорн...","[лераа, конфеты, внутри]","[наш, многодетный, семья, переехать, калифорни...","[лераа, конфета, внутри]","[америкацен, америка, ценывсша, ценывамерика, ...","[лераа, конфета, внутри]"
3,Наша многодетная семья переехала из Калифорнии...,http://www.youtube.com/watch?v=_UYt9yJFck0,_UYt9yJFck0,008c23470d1454e362218325921370f1,2023-11-05 00:59:59,NaN,"@gatta28учитесь общаться с людьми, не тыкайте....",http://www.youtube.com/watch?v=_UYt9yJFck0&lc=...,Ugy8QWttGacfEt3X0nl4AaABAg.9wiGws53R4J9wiYVObMGfF,4c772643e3ebbd01f0cf01cccdb402ba,2023-11-05 04:00:00,Ugy8QWttGacfEt3X0nl4AaABAg,"[наша, многодетная, семья, переехала, калифорн...","[gatta28учитесь, общаться, людьми, тыкайте, уч...","[наш, многодетный, семья, переехать, калифорни...","[gatta28учиться, общаться, человек, тыкать, уч...","[америкацен, америка, ценывсша, ценывамерика, ...","[gatta28учиться, общаться, человек, тыкать, уч..."
4,Наша многодетная семья переехала из Калифорнии...,http://www.youtube.com/watch?v=_UYt9yJFck0,_UYt9yJFck0,008c23470d1454e362218325921370f1,2023-11-05 00:59:59,NaN,Сама хожу с переломом мизинца. Это уже не перв...,http://www.youtube.com/watch?v=_UYt9yJFck0&lc=...,Ugy2NwZNhhWamuy3zMt4AaABAg,ae1d9b9b877b2c51787184d86e8ec843,2023-11-05 02:00:00,_UYt9yJFck0,"[наша, многодетная, семья, переехала, калифорн...","[сама, хожу, переломом, мизинца, это, первый, ...","[наш, многодетный, семья, переехать, калифорни...","[сам, ходить, перелом, мизинец, это, первый, р...","[америкацен, америка, ценывсша, ценывамерика, ...","[сам, ходить, мизинец, это, первый, раз, рентг..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352381,Вот смотрели зрители в ледовых шоу как Алекс...,https://dzen.ru/a/ZWxxfnv69RavXRXH,756471475437301336,04f5dd38e29f746fd188f640bf26fd80,2023-12-03 13:06:33,NaN,"Эх, Алнка))) родись она на год и 2 месяца пора...",https://dzen.ru/a/ZWxxfnv69RavXRXH#comment_193...,756471475437301336_1939824451,1e05fcc9c504173d30873db0baf239bd,2023-12-03 14:20:21,756471475437301336,"[смотрели, зрители, ледовых, шоу, александра, ...","[эх, алнка, р

In [41]:
#w2v
import gensim
from gensim.models import Word2Vec

sentences = words_x + words_y
model = gensim.models.Word2Vec(sentences, min_count=1)

In [49]:
model.wv[words_x[0][0]]

array([-0.0956148 ,  0.14434591,  0.18118003, -0.08206648, -0.10701546,
       -0.00601217,  0.17008953, -0.04191814,  0.05900566, -0.03462446,
       -0.19960769,  0.02160805,  0.02085581,  0.0974853 ,  0.046306  ,
       -0.08702892,  0.08584683, -0.12932166,  0.06724731, -0.11140703,
        0.33444706,  0.16043286, -0.0847922 ,  0.17423844,  0.2445722 ,
       -0.10597553,  0.07369604,  0.09538206, -0.02315765, -0.1181868 ,
        0.01136648, -0.12975055,  0.13025498, -0.12244213, -0.2627326 ,
       -0.06837352, -0.19513193,  0.05407478, -0.24287562, -0.11418078,
       -0.15239626, -0.1741293 , -0.0105749 , -0.16975904,  0.18236864,
        0.00830969, -0.03213952, -0.13050042,  0.09773064,  0.05480239,
        0.24866381,  0.04352815, -0.05522136, -0.10236616,  0.02713189,
        0.11406495,  0.02343059, -0.00250029, -0.03536095, -0.15439214,
       -0.03376802,  0.20972887, -0.10927313,  0.33434024, -0.01329539,
       -0.02618724,  0.08336997,  0.1567751 ,  0.21934046, -0.06

In [ ]:
#cosine_similarities for all vectors...

##Цель от оргов

In [ ]:
result = pd.read_json ('/content/ref_all_comments.json', lines=True)
result

,text,url,id,hash,date,comments,summary_string,summary_list
0,Как все изменилось(до конца)🤣,http://www.youtube.com/watch?v=JIOQYDNdS-k,JIOQYDNdS-k,003b1433b6d9a156db018c736daf02fe,2023-10-07 08:07:17,"[{'text': 'СТЭИРС, ГДЕ ТВОЙ ГОЛОС', 'url': 'ht...",изменения в голосе Стеирса и его возраст. Нек...,\n1. Изменение голоса у СТЭИРСА;\n2. Возвращен...
1,Видео архив с лайками и щенками и на этот раз ...,http://www.youtube.com/watch?v=DwyKDqipt9U,DwyKDqipt9U,004cec0fe5204393d658b70e7d1cd162,2023-10-07 15:46:31,"[{'text': '@JevgenijaKokorina-qs9oj Наоборот,...",рождение щенков у Златы и Белки. Некоторые лю...,\n1. Пополнение семьи щенков у Златы и Белки.\...
2,Всем привет! Сегодня встретилась с Таней и Кс...,http://www.youtube.com/watch?v=jHrKZm_T47E,jHrKZm_T47E,007a53ed25b012d37fb10253940e1b7a,2023-10-06 17:49:16,"[{'text': '☮️ ❤❤❤', 'url': 'http://www.youtube...","ароматы и парфюмерии, а также встречи с друзь...",\n1. Ароматы для женщин и коллекцию парфюмерии...
3,"Всем привет,мои дорогие друзья и гости моего к...",http://www.youtube.com/watch?v=nCE9SWn41vI,nCE9SWn41vI,00371d0fb29c76f9515ffb1ce2025cfc,2023-10-07 05:28:35,"[{'text': 'Да,Жуля всегда с нами!😁🥰🥰🥰', 'url':...",результаты забойки индюков и готовку их мяса....,\n1. Успешный забой индюков и полученные мясны...
4,"Вот и наступил момент, когда я накатал СВОИ пе...",http://www.youtube.com/watch?v=DzWQ0Sp0s5c,DzWQ0Sp0s5c,007f5c0c1a61734996e419c7f8c455ef,2023-10-05 17:23:23,[{'text': 'Ну 0 это просто повезло а так 30+ о...,"цены на новые автомобили, в том числе BMW 7-к...","\n1. Стоимость ТО на BMW 5ку и BMW 7ку, а такж..."
...,...,...,...,...,...,...,...,...
111,У нас на улицах уже осенний листопад 🕰️ 🍂 -Се...,https://telegram.me/olleysaas/728,1652332447_727,008ccd98956130e9807210228a7be9ad,2023-10-09 08:01:12,"[{'text': 'согласна#️⃣', 'url': 'https://teleg...",тему осеннего листопада и его красоту. Некото...,\n1. Красоту осеннего листопада и фотографий н...
112,Alfa Frontend Meetup: не забудьте прийти 🧚 На...,https://telegram.me/alfa_frontend/276,1813243978_276,014c8a1c0f9562f137f6fd853227720f,2023-10-09 08:33:36,"[{'text': 'Наконец-то полезный митап, на котор...",тему митапа Alfa Frontend Meetup и возможност...,\n1. Название митапа и дату его проведения.\n2...
113,Я часто говорю о ключевой роли вины и стыда у ...,https://telegram.me/amanasevich/1000,1262842820_1000,01a16799eea144b742b4a7254d6c35d4,2023-10-09 07:22:10,"[{'text': 'Почитаю 🔥', 'url': 'https://telegra...",тему зависимости и стыда у созависимых. Они о...,\n1. Роль вины и стыда у созависимых.\n2. Ощущ...
114,The Last of Us Part II не попадёт в каталог PS...,https://telegram.me/psprices_ru_ps4/13109,1046463780_13109,00c9c41c2f0c0b2455d9dc84051f8e57,2023-10-09 12:01:34,[{'text': 'Когда купил все эти игры и даром не...,The Last of Us Part II и PS Plus Extra. Некот...,"\n1. The Last of Us Part II и другие игры, кот..."


In [ ]:
result['summary_string'][1]

' рождение щенков у Златы и Белки. Некоторые люди поздравляли их с пополнением и просили здоровья малышам, другие выражали своё восхищение красотой щенков и хотели увидеть их рост. Некоторые люди также обсуждали проблемы содержания больших собак и пожелали Злате и Белке успехов в их работе.'

In [ ]:
result['comments'][1]

[{'text': '@JevgenijaKokorina-qs9oj\xa0 Наоборот, когда щенки на холоде, то у них подшёрсток растет лучше',
  'url': 'http://www.youtube.com/watch?v=DwyKDqipt9U&lc=UgyMDFRLZZPjVVXewrN4AaABAg.9v_NEc_uxx19veXm2Se-bC',
  'id': 'UgyMDFRLZZPjVVXewrN4AaABAg.9v_NEc_uxx19veXm2Se-bC',
  'hash': '450aeea379f03dc71b26704ad2f9da71',
  'root_id': 'DwyKDqipt9U',
  'parent_id': 'UgyMDFRLZZPjVVXewrN4AaABAg',
  'date': 1696876357},
 {'text': 'Поздравляем! Ждемс Чезку. Ох и детский сад будет ,табун носиться будет , вой, писк, веселуха 😄',
  'url': 'http://www.youtube.com/watch?v=DwyKDqipt9U&lc=UgyjK-jvO7vKfns8VaZ4AaABAg',
  'id': 'UgyjK-jvO7vKfns8VaZ4AaABAg',
  'hash': 'ec39de90cd6905763feeda7de31b0dba',
  'root_id': 'DwyKDqipt9U',
  'parent_id': 'DwyKDqipt9U',
  'date': 1696725474},
 {'text': 'Можно Вас поздравить 😊 Щенки красивые.Златка умничка ,пусть ростут здоровенькие .❤👍🐕Крым🐬',
  'url': 'http://www.youtube.com/watch?v=DwyKDqipt9U&lc=UgzMSkHBXYzcwS5vo7p4AaABAg',
  'id': 'UgzMSkHBXYzcwS5vo7p4AaABAg

In [ ]:
result['summary_list'][1]

'\n1. Пополнение семьи щенков у Златы и Белки.\n2. Уход за собаками, терпение и силы хозяина.\n3. Рождение щенков на седьмое октября.\n4. Стихия холода в Якутии.\n5. Поздравления с пополнением семьи.'

In [ ]:
result['text'][115]

'В новых Папиных Дочках есть стендап Как минимум в 9 серии, но скорее всего и в сериях раньше тоже, потому что персонаж этой девушки записалась на майк (прямо на месте, без плюсов и репостов) из-за своего парня стендап-комика. Если знаете, в каких сериях еще показывают стендап, напишите в комментах, покринжуем вместе'

In [ ]:
result['summary_string'][115]

' стендап-шоу в новых Папиных Дочках. Некоторые люди не могли понять, почему персонаж этой девушки записалась на майк из-за своего парня стендап-комика, а другие считали, что это был просто небольшой эпизод в сериале. Некоторые люди также обсуждали, как можно было бы улучшить стендап-шоу и привлечь больше зрителей к нему.'

In [ ]:
result['summary_list'][115]

'\n1. Стендап-комики и их качество. Некоторые люди не могли понять, почему эти комики вызывают интерес, а другие считали их забавными.\n2. Появление стендапа в новых Папиных Дочках. Некоторые люди не знали, что в сериях есть стендап, а другие уже видели его ранее.\n3. Как минимум в 9-й серии есть стендап, но скорее всего и в предыдущих сериях.\n4. В трепачах также есть стендап.'

In [ ]:
result['comments'][115]

[{'text': 'комики такие зануды я не могу, вам мало по жизни страданий с ебучими плюсиками ) еще на экране бы посмотреть',
  'url': 'https://telegram.me/comedy_for_300/1931?comment=6170',
  'id': '1558310898_6170',
  'hash': '3d2f1a24f87f90260f5f411c8b287f4c',
  'root_id': '1115928289_1931',
  'parent_id': '1115928289_1931',
  'date': 1696845804},
 {'text': 'Если ты доберешься до трепачей, то там максимально кринж) Как там показали все)',
  'url': 'https://telegram.me/comedy_for_300/1931?comment=6162',
  'id': '1558310898_6162',
  'hash': '801a24671254583e150a25be56508c06',
  'root_id': '1115928289_1931',
  'parent_id': '1115928289_1931',
  'date': 1696845378},
 {'text': 'Я не знаю что нужно сделать чтобы первых запомнили хотя бы',
  'url': 'https://telegram.me/comedy_for_300/1931?comment=6167',
  'id': '1558310898_6167',
  'hash': '9ec374c79c9978d43a077920f673105e',
  'root_id': '1115928289_1931',
  'parent_id': '1115928289_1931',
  'date': 1696845600},
 {'text': 'Да я не смотрю, чисто

In [ ]:
result['summary_list'][50]

'\n1. Возможные проблемы с автомобилем Voyah Free, такие как интеркуллер и батарея;\n2. Опыт владения электромобилем Voyah Free спустя год использования;\n3. Преимущества и недостатки электромобилей;\n4. Как ухаживать за электромобилем;\n5. Сервис и ремонт электромобиля;\n6. Возможность установки защиты днища для предотвращения повреждений от камней и снега;\n7. Отношение к автомобилю Voyah Free в целом, включая его достоинства и недостатки.'

In [ ]:
result['comments'][50]

[{'text': 'Авто крайне интересен ,но страшно ,наверное будц брать тпнк 500',
  'url': 'http://www.youtube.com/watch?v=ip32LMLdEbg&lc=Ugyqq-sMrsnGOUM3-lV4AaABAg',
  'id': 'Ugyqq-sMrsnGOUM3-lV4AaABAg',
  'hash': '04aaa33463d4755e87ab9d16078d69e5',
  'root_id': 'ip32LMLdEbg',
  'parent_id': 'ip32LMLdEbg',
  'date': 1698206400},
 {'text': '37000 болтается интеркуллер ,не видел видос где в гору но услышать за тачку в 5мл ехал в гору и авто перегрелось..почти.Контент крайне интересен -благодарю',
  'url': 'http://www.youtube.com/watch?v=ip32LMLdEbg&lc=UgyBahpmaatt_wEA1Mh4AaABAg',
  'id': 'UgyBahpmaatt_wEA1Mh4AaABAg',
  'hash': '4c21508241e3acb8061818a073c398de',
  'root_id': 'ip32LMLdEbg',
  'parent_id': 'ip32LMLdEbg',
  'date': 1698206400},
 {'text': 'ну потому что чаты как раз либо для флуда с мемами, либо когда уже прижмет. Эксплуатирую Voyah Free с середины мая, прошел первое ТО - никаких проблем ни по железу, ни по электронике, тьфу-тьфу. Авто берегу, по бездорожью без фанатизма. Ничто 